In [ ]:
import os
from nbed.driver import NbedDriver
import numpy as np

from nbed.ham_builder import HamiltonianBuilder
from openfermion import get_sparse_operator
import scipy as sp
    
from pyscf import gto, scf, fci, cc

In [ ]:
# get xyz file for water
notebook_dir = os.getcwd()
projects_dir = os.path.dirname(notebook_dir)
NBed_dir = os.path.dirname(projects_dir)
Test_dir = os.path.join(NBed_dir, 'tests')
mol_dir = os.path.join(Test_dir, 'molecules')

water_xyz_path = os.path.join(mol_dir, 'water.xyz')

In [ ]:
geometry_path = os.path.join(mol_dir, 'water.xyz')
# geometry_path = os.path.join(mol_dir, 'H4.xyz')

In [ ]:
geometry_path

In [ ]:
with open(geometry_path, 'r') as infile:
    xyz_string = infile.read()
    
print(xyz_string)

In [ ]:
# options
geometry = xyz_string
n_active_atoms=1
basis = 'STO-3G'
xc_functional = 'lda, vwn'# 'lda, vwn' #'B3LYP'
run_virtual_localization = False

run_fci_emb = True
run_ccsd_emb = False
max_ram_memory = 8_000
_init_huzinaga_rhf_with_mu = False

max_hf_cycles=5000


projector = 'both'
localization = 'ibo' # spade, ibo
occupied_threshold = 0.95
virtual_threshold = 0.95

In [ ]:
driver = NbedDriver(geometry = geometry,
                    n_active_atoms=n_active_atoms,
                    basis = basis,
                    xc_functional = xc_functional,
                    run_virtual_localization = run_virtual_localization,
                    run_fci_emb = run_fci_emb,
                    run_ccsd_emb = run_ccsd_emb,
                    max_ram_memory = max_ram_memory,
                    _init_huzinaga_rhf_with_mu = _init_huzinaga_rhf_with_mu,
                    max_hf_cycles=max_hf_cycles,
                    occupied_threshold=occupied_threshold,
                    projector = projector,
                    localization = localization)

In [ ]:
global_DFT_energy = driver._global_ks.e_tot
global_FCI_energy = driver._global_fci.e_tot

print(f'global FCI energy: {global_FCI_energy}')
print(f'global DFT energy: {global_DFT_energy}')

In [ ]:
qubit_transform = 'jordan_wigner'

    
huz_classical = driver._huzinaga['classical_energy']
rhf_huz = driver._huzinaga["scf"]
qham_huz = HamiltonianBuilder(
                        scf_method=rhf_huz,
                        constant_e_shift=huz_classical,
                        transform=qubit_transform,
                        ).build(n_qubits=None, taper=False)

mu_classical = driver._mu['classical_energy']
rhf_mu = driver._mu["scf"]
qham_mu = HamiltonianBuilder(
                        scf_method=rhf_mu,
                        constant_e_shift=mu_classical,
                        transform=qubit_transform,
                        ).build(n_qubits=None, taper=False)

In [ ]:
huz_H_sparse = get_sparse_operator(qham_huz)
huz_e_emb = sp.sparse.linalg.eigsh(huz_H_sparse, k=1, which='SA')[0][0]
huz_e_emb

mu_H_sparse = get_sparse_operator(qham_mu)
mu_e_emb = sp.sparse.linalg.eigsh(mu_H_sparse, k=1, which='SA')[0][0]
mu_e_emb

In [ ]:
huz_e_emb - driver._huzinaga['e_fci']

In [ ]:
mu_e_emb - driver._mu['e_fci']

In [ ]:
print(driver.localized_system.active_MO_inds)
print(driver.localized_system.enviro_MO_inds)

In [ ]:
print(f'global DFT error: {abs(global_FCI_energy - global_DFT_energy)}')

print(f'mu error: {abs(global_FCI_energy - huz_e_emb)}')
print(f'huz error: {abs(global_FCI_energy - mu_e_emb)}')

In [ ]:
# how subsystem was selected (either % active AOs [pyscf] or singular values[SPADE]!)
driver.localized_system.enviro_selection_condition

In [ ]:
len(list(qham_huz))

In [ ]:
len(list(qham_mu))

In [ ]:
driver._global_ks.e_tot

In [ ]:
driver.embed_dft_in_dft('B3LYP') # wb97m_v

In [ ]:
print(driver._mu_dft['e_rks_cheap'])
print(driver._mu_dft['e_rks_expen'])

In [ ]:
print(driver._huzinaga_dft['e_rks_cheap'])
print(driver._huzinaga_dft['e_rks_expen'])

In [ ]:
driver._global_ks.e_tot - driver._mu_dft['e_rks_cheap']

In [ ]:
driver._global_ks.e_tot - driver._huzinaga_dft['e_rks_cheap']

In [ ]:
mu_dft_in_dft_error =  abs(global_FCI_energy -  driver._mu_dft['e_rks_expen'])
print(f'mu error: {mu_dft_in_dft_error}')

huz_dft_in_dft_error =  abs(global_FCI_energy -  driver._huzinaga_dft['e_rks_expen'])
print(f'huz error: {huz_dft_in_dft_error}')

In [ ]:
cccc = driver._huzinaga_dft['scf_cheap_dft'].mo_coeff
FF = driver._huzinaga_dft['scf_cheap_dft'].get_fock()
np.around(cccc.T @ FF @ cccc, 2)

In [ ]:
driver._huzinaga_dft['scf_cheap_dft'].converged

In [ ]:
## can see which orbitals are removed (once env is removed)
# and their associated energies:

In [ ]:
print(driver._mu['mo_energies_emb_pre_del'])
print(driver._mu['mo_energies_emb_post_del'])

In [ ]:
print(driver._huzinaga['mo_energies_emb_pre_del'])
print(driver._huzinaga['mo_energies_emb_post_del'])